In [2]:
# The function localize takes the following arguments:
#
# colors:
#        2D list, each entry either 'R' (for red cell) or 'G' (for green cell)
#
# measurements:
#        list of measurements taken by the robot, each entry either 'R' or 'G'
#
# motions:
#        list of actions taken by the robot, each entry of the form [dy,dx],
#        where dx refers to the change in the x-direction (positive meaning
#        movement to the right) and dy refers to the change in the y-direction
#        (positive meaning movement downward)
#        NOTE: the *first* coordinate is change in y; the *second* coordinate is
#              change in x
#
# sensor_right:
#        float between 0 and 1, giving the probability that any given
#        measurement is correct; the probability that the measurement is
#        incorrect is 1-sensor_right
#
# p_move:
#        float between 0 and 1, giving the probability that any given movement
#        command takes place; the probability that the movement command fails
#        (and the robot remains still) is 1-p_move; the robot will NOT overshoot
#        its destination in this exercise
#
# The function should RETURN (not just show or print) a 2D list (of the same
# dimensions as colors) that gives the probabilities that the robot occupies
# each cell in the world.
#
# Compute the probabilities by assuming the robot initially has a uniform
# probability of being in any cell.
#
# Also assume that at each step, the robot:
# 1) first makes a movement,
# 2) then takes a measurement.
#
# Motion:
#  [0,0] - stay
#  [0,1] - right
#  [0,-1] - left
#  [1,0] - down
#  [-1,0] - up

def localize(colors,measurements,motions,sensor_right,p_move):
    # initializes p to a uniform distribution over a grid of the same dimensions as colors
    pinit = 1.0 / float(len(colors)) / float(len(colors[0]))
    p = [[pinit for row in range(len(colors[0]))] for col in range(len(colors))]

    
    # >>> Insert your code here <<<
    x_size = len(colors[0])
    y_size = len(colors)
    def sense(p, Z):
        q=[]
        for r in range(y_size):
            q.append([])
            for c in range(x_size):
                hit = (Z == colors[r][c])
                q[r].append(p[r][c] * (hit * sensor_right + (1-hit) * (1-sensor_right)))
        s = sum([sum(r) for r in q])
        for r in range(y_size):
            for c in range(x_size):
                q[r][c] /= s
        return q

    def move(p, U):
        q=[]
        for r in range(y_size):
            q.append([])
            for c in range(x_size):
                v = p[r][c] * (1-p_move)
                v += p[(r-U[0]) % y_size][(c-U[1]) % x_size] * p_move
                q[r].append(v)
        return q
        
    for Z,U in zip(measurements,motions):
        p = sense(move(p,U),Z)
    return p


def show(p):
    rows = ['[' + ','.join(map(lambda x: '{0:.5f}'.format(x),r)) + ']' for r in p]
    print '[' + ',\n '.join(rows) + ']'
    
#############################################################
# For the following test case, your output should be 
# [[0.01105, 0.02464, 0.06799, 0.04472, 0.02465],
#  [0.00715, 0.01017, 0.08696, 0.07988, 0.00935],
#  [0.00739, 0.00894, 0.11272, 0.35350, 0.04065],
#  [0.00910, 0.00715, 0.01434, 0.04313, 0.03642]]
# (within a tolerance of +/- 0.001 for each entry)

colors = [['R','G','G','R','R'],
          ['R','R','G','R','R'],
          ['R','R','G','G','R'],
          ['R','R','R','R','R']]
measurements = ['G','G','G','G','G']
motions = [[0,0],[0,1],[1,0],[1,0],[0,1]]
p = localize(colors,measurements,motions,sensor_right = 0.7, p_move = 0.8)
show(p) # displays your answer




[[0.01106,0.02464,0.06800,0.04472,0.02465],
 [0.00715,0.01017,0.08697,0.07988,0.00935],
 [0.00740,0.00894,0.11273,0.35351,0.04066],
 [0.00911,0.00715,0.01435,0.04313,0.03643]]


In [1]:
import sys
print(sys.version)
print(sys.path)

2.7.3 |Continuum Analytics, Inc.| (default, Dec 20 2012, 09:57:39) 
[GCC 4.0.1 (Apple Inc. build 5493)]
['', '/anaconda3/envs/py2/lib/python27.zip', '/anaconda3/envs/py2/lib/python2.7', '/anaconda3/envs/py2/lib/python2.7/plat-darwin', '/anaconda3/envs/py2/lib/python2.7/plat-mac', '/anaconda3/envs/py2/lib/python2.7/plat-mac/lib-scriptpackages', '/anaconda3/envs/py2/lib/python2.7/lib-tk', '/anaconda3/envs/py2/lib/python2.7/lib-old', '/anaconda3/envs/py2/lib/python2.7/lib-dynload', '/anaconda3/envs/py2/lib/python2.7/site-packages', '/anaconda3/envs/py2/lib/python2.7/site-packages/IPython/extensions', '/Users/pauldickinson/.ipython']


In [8]:
import math
mu = 10
sigma = 2
sigma_sq = sigma**2
f = lambda x, mu, sigma_sq:  math.exp(-0.5*(x-mu)**2 / sigma_sq) / math.sqrt(2*math.pi*sigma_sq)
f(8, mu, sigma_sq)

0.12098536225957168

In [10]:
update = lambda mu, sig_sq, nu, r_sq: ((r_sq*mu+sig_sq*nu)/(r_sq+sig_sq), 1./(1./sig_sq + 1./r_sq))

In [11]:
g(10,4,12,4)

(11, 2.0)

In [13]:
g(10.,8.,13.,2.)

(12.4, 1.6)

In [14]:
predict = lambda mean1, var1, mean2, var2: (mean1+mean2,var1+var2)

In [28]:
def update(prior,measure):
    mean1, var1 = prior
    mean2, var2 = measure
    new_mean = float(var2 * mean1 + var1 * mean2) / (var1 + var2)
    new_var = 1./(1./var1 + 1./var2)
    return [new_mean, new_var]

def predict(prior, move):
    mean1,var1 = prior
    mean2,var2 = move
    new_mean = mean1 + mean2
    new_var = var1 + var2
    return [new_mean, new_var]

measurements = [5., 6., 7., 9., 10.]
motion = [1., 1., 2., 1., 1.]
measurement_sig = 4.
motion_sig = 2.
mu = 0.
sig = 10000.

#Please print out ONLY the final values of the mean
#and the variance in a list [mu, sig]. 

# Insert code here
assert(len(measurements)==len(motion))

for z,u in zip(measurements,motion):
    mu,sig = update((mu,sig),(z,measurement_sig))
    mu,sig = predict((mu,sig),(u,motion_sig))
print [mu, sig]


[10.999906177177365, 4.005861580844194]


In [20]:
# import numpy as np
s = lambda measurement: (measurement, measurement_sig)
t = lambda motion: (motion, motion_sig)
list(zip(map(s, measurements), map(t,motion)))
# functools.reduce()

[((5.0, 4.0), (1.0, 2.0)),
 ((6.0, 4.0), (1.0, 2.0)),
 ((7.0, 4.0), (2.0, 2.0)),
 ((9.0, 4.0), (1.0, 2.0)),
 ((10.0, 4.0), (1.0, 2.0))]

In [32]:
mu = 0.
sig = 10000.
for z,u in zip(map(s, measurements), map(t,motion)):
    mu,sig = predict((update((mu,sig),z)),u)
print (mu,sig)

(10.999906177177365, 4.005861580844194)


In [34]:
[(mu,sig)] + zip(map(s, measurements), map(t,motion))

[(10.999906177177365, 4.005861580844194),
 ((5.0, 4.0), (1.0, 2.0)),
 ((6.0, 4.0), (1.0, 2.0)),
 ((7.0, 4.0), (2.0, 2.0)),
 ((9.0, 4.0), (1.0, 2.0)),
 ((10.0, 4.0), (1.0, 2.0))]

In [38]:
mu,sig = 0.,10000.
reduce(
    lambda prior, (z,u): predict(update(prior,z),u),
    [(mu,sig)]+zip(map(s, measurements), map(t,motion))
)

[10.999906177177365, 4.005861580844194]

In [43]:
# Write a function 'kalman_filter' that implements a multi-
# dimensional Kalman Filter for the example given

from math import *


class matrix:
    
    # implements basic operations of a matrix class
    
    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0
    
    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            raise ValueError, "Invalid size of matrix"
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]
    
    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
            raise ValueError, "Invalid size of matrix"
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1
    
    def show(self):
        for i in range(self.dimx):
            print(self.value[i])
        print(' ')
    
    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise ValueError, "Matrices must be of equal dimensions to add"
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res
    
    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise ValueError, "Matrices must be of equal dimensions to subtract"
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res
    
    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
            raise ValueError, "Matrices must be m*n and n*p to multiply"
        else:
            # multiply if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res
    
    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res
    
    # Thanks to Ernesto P. Adorio for use of Cholesky and CholeskyInverse functions
    
    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                    raise ValueError, "Matrix not positive-definite"
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                try:
                   res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
                except:
                   raise ValueError, "Zero diagonal"
        return res
    
    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res
    
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res
    
    def __repr__(self):
        return repr(self.value)


########################################

# Implement the filter function below

def kalman_filter(x, P):
    for n in range(len(measurements)):
        
        # measurement update
        x,P = update(x,P,measurements[n])
        # prediction
        x,P = predict(x,P)
        
    return x,P
    
    
def predict(x, P):
    xNew = F * x + u
    PNew = F * P * F.transpose()
    return xNew, PNew

def update(x, P, z):
    y = matrix([[z]]) - H * x
    S = H * P * H.transpose() + R
    K = P * H.transpose() * S.inverse()
    xNew = x + (K * y)
    PNew = (I - K * H) * P
    return xNew, PNew

############################################
### use the code below to test your filter!
############################################

measurements = [1, 2, 3]

x = matrix([[0.], [0.]]) # initial state (location and velocity)
P = matrix([[1000., 0.], [0., 1000.]]) # initial uncertainty
u = matrix([[0.], [0.]]) # external motion
F = matrix([[1., 1.], [0, 1.]]) # next state function
H = matrix([[1., 0.]]) # measurement function
R = matrix([[1.]]) # measurement uncertainty
I = matrix([[1., 0.], [0., 1.]]) # identity matrix



print(kalman_filter(x, P))
# output should be:
# x: [[3.9996664447958645], [0.9999998335552873]]
# P: [[2.3318904241194827, 0.9991676099921091], [0.9991676099921067, 0.49950058263974184]]


[[0.0]]
[[0.0]]
([[3.9996664447958645], [0.9999998335552873]], [[2.3318904241194827, 0.9991676099921091], [0.9991676099921067, 0.49950058263974184]])


# Asst 2 -- Kalman Filters

In [1]:
# Fill in the matrices P, F, H, R and I at the bottom
#
# This question requires NO CODING, just fill in the 
# matrices where indicated. Please do not delete or modify
# any provided code OR comments. Good luck!

from math import *

class matrix:
    
    # implements basic operations of a matrix class
    
    def __init__(self, value):
        self.value = value
        self.dimx = len(value)
        self.dimy = len(value[0])
        if value == [[]]:
            self.dimx = 0
    
    def zero(self, dimx, dimy):
        # check if valid dimensions
        if dimx < 1 or dimy < 1:
            raise ValueError, "Invalid size of matrix"
        else:
            self.dimx = dimx
            self.dimy = dimy
            self.value = [[0 for row in range(dimy)] for col in range(dimx)]
    
    def identity(self, dim):
        # check if valid dimension
        if dim < 1:
            raise ValueError, "Invalid size of matrix"
        else:
            self.dimx = dim
            self.dimy = dim
            self.value = [[0 for row in range(dim)] for col in range(dim)]
            for i in range(dim):
                self.value[i][i] = 1
    
    def show(self):
        for i in range(self.dimx):
            print self.value[i]
        print ' '
    
    def __add__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise ValueError, "Matrices must be of equal dimensions to add"
        else:
            # add if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] + other.value[i][j]
            return res
    
    def __sub__(self, other):
        # check if correct dimensions
        if self.dimx != other.dimx or self.dimy != other.dimy:
            raise ValueError, "Matrices must be of equal dimensions to subtract"
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, self.dimy)
            for i in range(self.dimx):
                for j in range(self.dimy):
                    res.value[i][j] = self.value[i][j] - other.value[i][j]
            return res
    
    def __mul__(self, other):
        # check if correct dimensions
        if self.dimy != other.dimx:
            raise ValueError, "Matrices must be m*n and n*p to multiply"
        else:
            # subtract if correct dimensions
            res = matrix([[]])
            res.zero(self.dimx, other.dimy)
            for i in range(self.dimx):
                for j in range(other.dimy):
                    for k in range(self.dimy):
                        res.value[i][j] += self.value[i][k] * other.value[k][j]
            return res
    
    def transpose(self):
        # compute transpose
        res = matrix([[]])
        res.zero(self.dimy, self.dimx)
        for i in range(self.dimx):
            for j in range(self.dimy):
                res.value[j][i] = self.value[i][j]
        return res
    
    # Thanks to Ernesto P. Adorio for use of Cholesky and CholeskyInverse functions
    
    def Cholesky(self, ztol=1.0e-5):
        # Computes the upper triangular Cholesky factorization of
        # a positive definite matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        for i in range(self.dimx):
            S = sum([(res.value[k][i])**2 for k in range(i)])
            d = self.value[i][i] - S
            if abs(d) < ztol:
                res.value[i][i] = 0.0
            else:
                if d < 0.0:
                    raise ValueError, "Matrix not positive-definite"
                res.value[i][i] = sqrt(d)
            for j in range(i+1, self.dimx):
                S = sum([res.value[k][i] * res.value[k][j] for k in range(self.dimx)])
                if abs(S) < ztol:
                    S = 0.0
                try:
                   res.value[i][j] = (self.value[i][j] - S)/res.value[i][i]
                except:
                   raise ValueError, "Zero diagonal"
        return res
    
    def CholeskyInverse(self):
        # Computes inverse of matrix given its Cholesky upper Triangular
        # decomposition of matrix.
        res = matrix([[]])
        res.zero(self.dimx, self.dimx)
        
        # Backward step for inverse.
        for j in reversed(range(self.dimx)):
            tjj = self.value[j][j]
            S = sum([self.value[j][k]*res.value[j][k] for k in range(j+1, self.dimx)])
            res.value[j][j] = 1.0/tjj**2 - S/tjj
            for i in reversed(range(j)):
                res.value[j][i] = res.value[i][j] = -sum([self.value[i][k]*res.value[k][j] for k in range(i+1, self.dimx)])/self.value[i][i]
        return res
    
    def inverse(self):
        aux = self.Cholesky()
        res = aux.CholeskyInverse()
        return res
    
    def __repr__(self):
        return repr(self.value)


########################################

def filter(x, P):
    for n in range(len(measurements)):
        
        # prediction
        x = (F * x) + u
        P = F * P * F.transpose()
        
        # measurement update
        Z = matrix([measurements[n]])
        # print(Z)
        # print(H)
        # print(x)
        y = Z.transpose() - (H * x)
        S = H * P * H.transpose() + R
        K = P * H.transpose() * S.inverse()
        x = x + (K * y)
        P = (I - (K * H)) * P
    
    print 'x= '
    x.show()
    print 'P= '
    P.show()

########################################

print "### 4-dimensional example ###"

measurements = [[5., 10.], [6., 8.], [7., 6.], [8., 4.], [9., 2.], [10., 0.]]
initial_xy = [4., 12.]

# measurements = [[1., 4.], [6., 0.], [11., -4.], [16., -8.]]
# initial_xy = [-4., 8.]

# measurements = [[1., 17.], [1., 15.], [1., 13.], [1., 11.]]
# initial_xy = [1., 19.]

dt = 0.1

x = matrix([[initial_xy[0]], [initial_xy[1]], [0.], [0.]]) # initial state (location and velocity)
u = matrix([[0.], [0.], [0.], [0.]]) # external motion

#### DO NOT MODIFY ANYTHING ABOVE HERE ####
#### fill this in, remember to use the matrix() function!: ####

P =  matrix([[0.,0.,0.,0.,],[0.,0.,0.,0.,],[0.,0.,1000.,0.],[0.,0.,0.,1000.]]) # initial uncertainty: 0 for positions x and y, 1000 for the two velocities
F =  matrix([[1.,0.,dt,0.],[0.,1.,0.,dt],[0.,0.,1.,0.],[0.,0.,0.,1.]])# next state function: generalize the 2d version to 4d
H =  matrix([[1.,0.,0.,0.],[0.,1.,0.,0.]])# measurement function: reflect the fact that we observe x and y but not the two velocities
R =  matrix([[0.1,0.],[0.,0.1]])# measurement uncertainty: use 2x2 matrix with 0.1 as main diagonal
I =  matrix([[1.,0.,0.,0.],[0.,1.,0.,0.],[0.,0.,1.,0.],[0.,0.,0.,1.]])# 4d identity matrix

###### DO NOT MODIFY ANYTHING HERE #######

filter(x, P)


### 4-dimensional example ###
x= 
[9.999340731787717]
[0.001318536424568617]
[9.998901219646193]
[-19.997802439292386]
 
P= 
[0.03955609273706198, 0.0, 0.06592682122843721, 0.0]
[0.0, 0.03955609273706198, 0.0, 0.06592682122843721]
[0.06592682122843718, 0.0, 0.10987803538073201, 0.0]
[0.0, 0.06592682122843718, 0.0, 0.10987803538073201]
 


# Part 3 -- Particle Filters

In [2]:
# Make a robot called myrobot that starts at
# coordinates 30, 50 heading north (pi/2).
# Have your robot turn clockwise by pi/2, move
# 15 m, and sense. Then have it turn clockwise
# by pi/2 again, move 10 m, and sense again.
#
# Your program should print out the result of
# your two sense measurements.
#
# Don't modify the code below. Please enter
# your code at the bottom.

from math import *
import random



landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0


class robot:
    def __init__(self):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0;
        self.turn_noise    = 0.0;
        self.sense_noise   = 0.0;
    
    def set(self, new_x, new_y, new_orientation):
        if new_x < 0 or new_x >= world_size:
            raise ValueError, 'X coordinate out of bound'
        if new_y < 0 or new_y >= world_size:
            raise ValueError, 'Y coordinate out of bound'
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError, 'Orientation must be in [0..2pi]'
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
    
    
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.forward_noise = float(new_f_noise);
        self.turn_noise    = float(new_t_noise);
        self.sense_noise   = float(new_s_noise);
    
    
    def sense(self):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    
    def move(self, turn, forward):
        if forward < 0:
            raise ValueError, 'Robot cant move backwards'         
        
        # turn, and add randomness to the turning command
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        # move, and add randomness to the motion command
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size    # cyclic truncate
        y %= world_size
        
        # set particle
        res = robot()
        res.set(x, y, orientation)
        res.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return res
    
    def Gaussian(self, mu, sigma, x):
        
        # calculates the probability of x for 1-dim Gaussian with mean mu and var. sigma
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))
    
    
    def measurement_prob(self, measurement):
        
        # calculates how likely a measurement should be
        
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))



def eval(r, p):
    sum = 0.0;
    for i in range(len(p)): # calculate mean error
        dx = (p[i].x - r.x + (world_size/2.0)) % world_size - (world_size/2.0)
        dy = (p[i].y - r.y + (world_size/2.0)) % world_size - (world_size/2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))



####   DON'T MODIFY ANYTHING ABOVE HERE! ENTER CODE BELOW ####




In [4]:
# Make a robot called myrobot that starts at
# coordinates 30, 50 heading north (pi/2).
# Have your robot turn clockwise by pi/2, move
# 15 m, and sense. Then have it turn clockwise
# by pi/2 again, move 10 m, and sense again.
#
# Your program should print out the result of
# your two sense measurements.
#
# Don't modify the code below. Please enter
# your code at the bottom.


myrobot = robot()
myrobot.set(30.,50.,pi/2.)
myrobot = myrobot.move(-pi/2.,15.)
print(myrobot.sense())
myrobot = myrobot.move(-pi/2.,10.)
print(myrobot.sense())

[39.05124837953327, 46.09772228646444, 39.05124837953327, 46.09772228646444]
[32.01562118716424, 53.150729063673246, 47.16990566028302, 40.311288741492746]


In [6]:
# Now add noise to your robot as follows:
# forward_noise = 5.0, turn_noise = 0.1,
# sense_noise = 5.0.
myrobot = robot()
myrobot.set_noise(5.,0.1,5.)
myrobot.set(30.,50.,pi/2.)
myrobot = myrobot.move(-pi/2.,15.)
print(myrobot.sense())
myrobot = myrobot.move(-pi/2.,10.)
print(myrobot.sense())

[48.658004340440684, 42.3584925130414, 43.92906587106809, 44.78061547193783]
[39.94386130727415, 45.754690869502255, 53.65486005798247, 28.27059163903773]


In [9]:
# Make 1000 robots (particles)
n_robots = 1000
def noisy_robot(f,t,s):
    r = robot()
    r.set_noise(f,t,s)
    return r

p = [noisy_robot(0.05,0.05,5.) for i in range(n_robots)]



In [13]:
# Make the agent, move, and sense
myrobot = robot()
myrobot = myrobot.move(0.1, 5.0)
Z = myrobot.sense()



In [53]:
# weight the particles by the likelihood that they match the sensed environment
p = [r.move(0.1,5.) for r in p]
w = [r.measurement_prob(Z) for r in p]

In [39]:
# resample (O(n) solution is appropriate for 1 sample, but for large numbers should be smarter)
def weighted_choice(choices):
    total = sum(w for c, w in choices)    
    r = random.uniform(0., total)
    upto = 0.
    for c, w in choices:
        if upto + w >= r:
            return c
        upto += w
    assert False, "Shouldn't get here"

w_c = sorted(list(enumerate(w)),key=lambda tup:-tup[1]) # sorting makes it faster
q = [p[weighted_choice(w_c)] for _ in range(n_robots)]

In [65]:
# alternative resample based on "wheel" approach from lecture
import random
def resample_n(w,N):
    wl = len(w)
    wm = max(w)
    ix = random.randint(0,wl-1)
    beta = 0.
    r = []
    for j in range(N):
        beta += random.uniform(0.,2.*wm)
        while beta > w[ix]:
            beta -= w[ix]
            ix = (ix+1)%wl            
        r.append(ix)
    return r

In [87]:
# Initialize
myrobot = robot()
p = [noisy_robot(0.05,0.05,5.) for i in range(n_robots)]
for r in p:
    print r


[x=76.981 y=80.107 orient=0.0794]
[x=93.428 y=23.849 orient=2.6847]
[x=37.962 y=42.391 orient=2.1472]
[x=91.689 y=82.637 orient=2.3175]
[x=13.588 y=50.100 orient=4.1116]
[x=38.382 y=23.166 orient=5.6464]
[x=17.251 y=69.798 orient=1.4349]
[x=14.234 y=81.897 orient=1.8541]
[x=3.4613 y=54.983 orient=2.9971]
[x=74.618 y=20.139 orient=4.9289]
[x=55.692 y=20.014 orient=2.2065]
[x=36.376 y=91.101 orient=5.8700]
[x=47.850 y=68.312 orient=3.5864]
[x=27.639 y=60.725 orient=1.4893]
[x=58.210 y=40.784 orient=6.0528]
[x=15.821 y=58.094 orient=0.0799]
[x=63.941 y=14.124 orient=0.8180]
[x=30.698 y=36.311 orient=6.0458]
[x=63.345 y=53.536 orient=4.9869]
[x=97.138 y=68.677 orient=3.0714]
[x=27.819 y=90.474 orient=0.7696]
[x=44.853 y=99.329 orient=2.5341]
[x=22.241 y=2.9242 orient=6.2728]
[x=31.096 y=79.555 orient=0.4657]
[x=28.180 y=7.9695 orient=2.7480]
[x=86.982 y=41.692 orient=5.6484]
[x=73.109 y=44.207 orient=1.9421]
[x=54.368 y=25.610 orient=3.1899]
[x=28.751 y=37.539 orient=6.2723]
[x=26.058 y=57

In [88]:
# Move and filter
myrobot.move(0.1,5.)
Z = myrobot.sense()
p = [r.move(0.1,5.) for r in p]
w = [r.measurement_prob(Z) for r in p]
p = [p[i] for i in resample_n(w,1000)]
for r in p:
    print r

[x=10.506 y=61.243 orient=4.3333]
[x=7.3688 y=70.908 orient=4.9615]
[x=15.214 y=66.771 orient=1.2125]
[x=11.875 y=64.542 orient=4.2055]
[x=6.1348 y=63.249 orient=5.4560]
[x=5.7843 y=68.109 orient=2.2034]
[x=12.275 y=71.047 orient=3.2247]
[x=14.575 y=80.364 orient=5.3533]
[x=6.4217 y=67.290 orient=2.0303]
[x=17.567 y=69.384 orient=0.7035]
[x=7.3688 y=70.908 orient=4.9615]
[x=5.6430 y=60.697 orient=3.9692]
[x=15.018 y=63.992 orient=4.5865]
[x=12.922 y=65.588 orient=4.1857]
[x=8.9062 y=62.362 orient=2.7402]
[x=10.828 y=59.710 orient=5.7755]
[x=12.275 y=71.047 orient=3.2247]
[x=12.275 y=71.047 orient=3.2247]
[x=12.275 y=71.047 orient=3.2247]
[x=6.4217 y=67.290 orient=2.0303]
[x=10.517 y=65.348 orient=0.0326]
[x=10.517 y=65.348 orient=0.0326]
[x=15.214 y=66.771 orient=1.2125]
[x=15.214 y=66.771 orient=1.2125]
[x=11.875 y=64.542 orient=4.2055]
[x=14.913 y=81.971 orient=2.8702]
[x=6.1348 y=63.249 orient=5.4560]
[x=12.922 y=65.588 orient=4.1857]
[x=12.922 y=65.588 orient=4.1857]
[x=12.922 y=65

In [89]:
# Move and filter again...
myrobot.move(0.1,5.)
Z = myrobot.sense()
p = [r.move(0.1,5.) for r in p]
w = [r.measurement_prob(Z) for r in p]
p = [p[i] for i in resample_n(w,1000)]
for r in p:
    print r

[x=15.521 y=66.019 orient=0.1333]
[x=10.798 y=61.127 orient=4.2681]
[x=7.3395 y=70.268 orient=3.2980]
[x=5.4850 y=67.753 orient=2.1076]
[x=5.4850 y=67.753 orient=2.1076]
[x=9.3709 y=66.234 orient=5.1171]
[x=9.1941 y=66.252 orient=5.0859]
[x=2.7815 y=67.707 orient=3.0482]
[x=14.513 y=63.981 orient=3.9109]
[x=5.5303 y=67.827 orient=2.0924]
[x=5.5303 y=67.827 orient=2.0924]
[x=9.2019 y=66.303 orient=5.0912]
[x=5.4381 y=67.707 orient=2.1204]
[x=13.047 y=68.199 orient=3.8671]
[x=11.335 y=60.839 orient=4.3898]
[x=9.2461 y=66.239 orient=5.0946]
[x=9.7110 y=59.964 orient=4.2708]
[x=3.7310 y=71.606 orient=2.1282]
[x=8.9280 y=66.182 orient=5.0310]
[x=7.5043 y=69.835 orient=3.3902]
[x=7.5043 y=69.835 orient=3.3902]
[x=2.8334 y=67.884 orient=3.0120]
[x=15.555 y=65.819 orient=0.0931]
[x=9.0615 y=66.223 orient=5.0590]
[x=9.0615 y=66.223 orient=5.0590]
[x=5.1983 y=72.676 orient=1.5025]
[x=13.145 y=68.174 orient=3.8841]
[x=10.723 y=61.183 orient=4.2493]
[x=9.1737 y=66.225 orient=5.0802]
[x=9.1737 y=66

In [100]:
# Do it T times
myrobot = robot()
n_robots = 10000
p = [noisy_robot(0.05,0.05,5.) for i in range(n_robots)]
print(eval(myrobot,p))

T = 10
for t in range(T):
    m = (0.1,5.0)
    myrobot = myrobot.move(*m)
    Z = myrobot.sense()
    p = [r.move(*m) for r in p]
    w = [r.measurement_prob(Z) for r in p]
    p = [p[i] for i in resample_n(w,len(p))]
    print(eval(myrobot,p))

# myrobot,p = move_robot(myrobot,p,0.1,5.0)
# myrobot,p = move_robot(myrobot,p,0.1,5.0)         

38.3352325439
5.4146381472
5.09277861096
4.5509772234
2.53650539452
2.0699856049
2.1391969908
2.26613760752
2.23047743804
2.13914560379
2.0540359756


Recall, this is just:
measurement update:
P(X|Z) = c * P(Z|X) * P(X)
(c is normalization constant)

motion updates:
P(X') = sum(P(X'|X) * P(X))
this is a convolution over all possible positions X

Bicycle motion model:

Original orientation: $\theta$

Turning angle: $\beta = \frac{d}{L} . \tan(\alpha)$

Turning radius: $R = \frac{d}{\beta}$

Center of turning $(cx,cy)$: 

$cx = x - \sin(\theta) . R$

$cy = y + \cos(\theta) . R$

New position:
$x' = cx + \sin(\theta+\beta) . R$

$y' = cy - \cos(\theta+\beta) . R$

$\theta' = \theta + \beta$

For small $\beta$, we approximate with straight motion:

$x' = x + d . \cos(\theta)$

$y' = y + d . \sin(\theta)$

$\theta' = \theta$

In [111]:
# -----------------
# USER INSTRUCTIONS
#
# Write a function in the class robot called move()
#
# that takes self and a motion vector (this
# motion vector contains a steering* angle and a
# distance) as input and returns an instance of the class
# robot with the appropriate x, y, and orientation
# for the given motion.
#
# *steering is defined in the video
# which accompanies this problem.
#
# For now, please do NOT add noise to your move function.
#
# Please do not modify anything except where indicated
# below.
#
# There are test cases which you are free to use at the
# bottom. If you uncomment them for testing, make sure you
# re-comment them before you submit.

from math import *
import random
# --------
# 
# the "world" has 4 landmarks.
# the robot's initial coordinates are somewhere in the square
# represented by the landmarks.
#
# NOTE: Landmark coordinates are given in (y, x) form and NOT
# in the traditional (x, y) format!

landmarks  = [[0.0, 100.0], [0.0, 0.0], [100.0, 0.0], [100.0, 100.0]] # position of 4 landmarks
world_size = 100.0 # world is NOT cyclic. Robot is allowed to travel "out of bounds"
max_steering_angle = pi/4 # You don't need to use this value, but it is good to keep in mind the limitations of a real car.

# ------------------------------------------------
# 
# this is the robot class
#

class robot:

    # --------

    # init: 
    #	creates robot and initializes location/orientation 
    #

    def __init__(self, length = 10.0):
        self.x = random.random() * world_size # initial x position
        self.y = random.random() * world_size # initial y position
        self.orientation = random.random() * 2.0 * pi # initial orientation
        self.length = length # length of robot
        self.bearing_noise  = 0.0 # initialize bearing noise to zero
        self.steering_noise = 0.0 # initialize steering noise to zero
        self.distance_noise = 0.0 # initialize distance noise to zero
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))
    # --------
    # set: 
    #	sets a robot coordinate
    #

    def set(self, new_x, new_y, new_orientation):

        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError, 'Orientation must be in [0..2pi]'
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)


    # --------
    # set_noise: 
    #	sets the noise parameters
    #

    def set_noise(self, new_b_noise, new_s_noise, new_d_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.bearing_noise  = float(new_b_noise)
        self.steering_noise = float(new_s_noise)
        self.distance_noise = float(new_d_noise)
    
    ############# ONLY ADD/MODIFY CODE BELOW HERE ###################

    # --------
    # move:
    #   move along a section of a circular path according to motion
    #
    
    def move(self, motion): # Do not change the name of this function

        # ADD CODE HERE

        epsilon = 0.0001
        x,y,theta = self.x,self.y,self.orientation
        alpha, d = motion
        alpha += random.gauss(0,self.steering_noise)
        if abs(alpha) > max_steering_angle:
            raise ValueError, "Exceeded max steering angle"
            
        if d < 0.0:
            raise ValueError, "Can't move backwards"
        

        beta = d / self.length * tan(alpha)

        
        if abs(beta) > epsilon:
            R = d / beta
            cx = x - sin(theta)*R
            cy = y + cos(theta)*R
            x = cx + sin(theta + beta)*R
            y = cy - cos(theta + beta)*R

        else:
            x = x + d * cos(theta)
            y = y + d * sin(theta)

        theta = (theta + beta) % (2*pi)
        result = robot(self.length)
        result.set(x,y,theta)
        
        return result # make sure your move function returns an instance
                      # of the robot class with the correct coordinates.
                      
    ############## ONLY ADD/MODIFY CODE ABOVE HERE ####################
        

## IMPORTANT: You may uncomment the test cases below to test your code.
## But when you submit this code, your test cases MUST be commented
## out. Our testing program provides its own code for testing your
## move function with randomized motion data.

## --------
## TEST CASE:
## 
## 1) The following code should print:
##       Robot:     [x=0.0 y=0.0 orient=0.0]
##       Robot:     [x=10.0 y=0.0 orient=0.0]
##       Robot:     [x=19.861 y=1.4333 orient=0.2886]
##       Robot:     [x=39.034 y=7.1270 orient=0.2886]
##
##
length = 20.
bearing_noise  = 0.0
steering_noise = 0.0
distance_noise = 0.0

myrobot = robot(length)
myrobot.set(0.0, 0.0, 0.0)
myrobot.set_noise(bearing_noise, steering_noise, distance_noise)

motions = [[0.0, 10.0], [pi / 6.0, 10], [0.0, 20.0]]

T = len(motions)

print 'Robot:    ', myrobot
for t in range(T):
    myrobot = myrobot.move(motions[t])
    print 'Robot:    ', myrobot



## IMPORTANT: You may uncomment the test cases below to test your code.
## But when you submit this code, your test cases MUST be commented
## out. Our testing program provides its own code for testing your
## move function with randomized motion data.

    
## 2) The following code should print:
##      Robot:     [x=0.0 y=0.0 orient=0.0]
##      Robot:     [x=9.9828 y=0.5063 orient=0.1013]
##      Robot:     [x=19.863 y=2.0201 orient=0.2027]
##      Robot:     [x=29.539 y=4.5259 orient=0.3040]
##      Robot:     [x=38.913 y=7.9979 orient=0.4054]
##      Robot:     [x=47.887 y=12.400 orient=0.5067]
##      Robot:     [x=56.369 y=17.688 orient=0.6081]
##      Robot:     [x=64.273 y=23.807 orient=0.7094]
##      Robot:     [x=71.517 y=30.695 orient=0.8108]
##      Robot:     [x=78.027 y=38.280 orient=0.9121]
##      Robot:     [x=83.736 y=46.485 orient=1.0135]
##
##
##length = 20.
##bearing_noise  = 0.0
##steering_noise = 0.0
##distance_noise = 0.0
##
##myrobot = robot(length)
##myrobot.set(0.0, 0.0, 0.0)
##myrobot.set_noise(bearing_noise, steering_noise, distance_noise)
##
##motions = [[0.2, 10.] for row in range(10)]
##
##T = len(motions)
##
##print 'Robot:    ', myrobot
##for t in range(T):
##    myrobot = myrobot.move(motions[t])
##    print 'Robot:    ', myrobot

## IMPORTANT: You may uncomment the test cases below to test your code.
## But when you submit this code, your test cases MUST be commented
## out. Our testing program provides its own code for testing your
## move function with randomized motion data.




Robot:     [x=0.0 y=0.0 orient=0.0]
Robot:     [x=10.0 y=0.0 orient=0.0]
Robot:     [x=19.861 y=1.4333 orient=0.2886]
Robot:     [x=39.034 y=7.1270 orient=0.2886]


In [110]:
# --------------
# USER INSTRUCTIONS
#
# Write a function in the class robot called sense()
# that takes self as input
# and returns a list, Z, of the four bearings* to the 4
# different landmarks. you will have to use the robot's
# x and y position, as well as its orientation, to
# compute this.
#
# *bearing is defined in the video
# which accompanies this problem.
#
# For now, please do NOT add noise to your sense function.
#
# Please do not modify anything except where indicated
# below.
#
# There are test cases provided at the bottom which you are
# free to use. If you uncomment any of these cases for testing
# make sure that you re-comment it before you submit.

from math import *
import random

# --------
# 
# the "world" has 4 landmarks.
# the robot's initial coordinates are somewhere in the square
# represented by the landmarks.
#
# NOTE: Landmark coordinates are given in (y, x) form and NOT
# in the traditional (x, y) format!

landmarks  = [[0.0, 100.0], [0.0, 0.0], [100.0, 0.0], [100.0, 100.0]] # position of 4 landmarks in (y, x) form.
world_size = 100.0 # world is NOT cyclic. Robot is allowed to travel "out of bounds"

# ------------------------------------------------
# 
# this is the robot class
#

class robot:

    # --------
    # init: 
    #	creates robot and initializes location/orientation
    #

    def __init__(self, length = 10.0):
        self.x = random.random() * world_size # initial x position
        self.y = random.random() * world_size # initial y position
        self.orientation = random.random() * 2.0 * pi # initial orientation
        self.length = length # length of robot
        self.bearing_noise  = 0.0 # initialize bearing noise to zero
        self.steering_noise = 0.0 # initialize steering noise to zero
        self.distance_noise = 0.0 # initialize distance noise to zero



    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), 
                                                str(self.orientation))


    # --------
    # set: 
    #	sets a robot coordinate
    #

    def set(self, new_x, new_y, new_orientation):
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError, 'Orientation must be in [0..2pi]'
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)

    # --------
    # set_noise: 
    #	sets the noise parameters
    #

    def set_noise(self, new_b_noise, new_s_noise, new_d_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.bearing_noise  = float(new_b_noise)
        self.steering_noise = float(new_s_noise)
        self.distance_noise = float(new_d_noise)

    ############# ONLY ADD/MODIFY CODE BELOW HERE ###################

    # --------
    # sense:
    #   obtains bearings from positions
    #
    
    def sense(self): #do not change the name of this function
        # Z = []

        # ENTER CODE HERE
        # HINT: You will probably need to use the function atan2()
        Z = [(atan2(
                landmark[0]-self.y,
                landmark[1]-self.x
            ) - self.orientation + random.gauss(0,self.bearing_noise)) % (2*pi)
             for landmark in landmarks]

        return Z #Leave this line here. Return vector Z of 4 bearings.
    
    ############## ONLY ADD/MODIFY CODE ABOVE HERE ####################


## IMPORTANT: You may uncomment the test cases below to test your code.
## But when you submit this code, your test cases MUST be commented
## out. Our testing program provides its own code for testing your
## sense function with randomized initial robot coordinates.
    
## --------
## TEST CASES:



##
## 1) The following code should print the list [6.004885648174475, 3.7295952571373605, 1.9295669970654687, 0.8519663271732721]
##

length = 20.
bearing_noise  = 0.0
steering_noise = 0.0
distance_noise = 0.0

myrobot = robot(length)
myrobot.set(30.0, 20.0, 0.0)
myrobot.set_noise(bearing_noise, steering_noise, distance_noise)

print 'Robot:        ', myrobot
print 'Measurements: ', myrobot.sense()


## IMPORTANT: You may uncomment the test cases below to test your code.
## But when you submit this code, your test cases MUST be commented
## out. Our testing program provides its own code for testing your
## sense function with randomized initial robot coordinates.
    

##
## 2) The following code should print the list [5.376567117456516, 3.101276726419402, 1.3012484663475101, 0.22364779645531352]
##
##
##length = 20.
##bearing_noise  = 0.0
##steering_noise = 0.0
##distance_noise = 0.0
##
##myrobot = robot(length)
##myrobot.set(30.0, 20.0, pi / 5.0)
##myrobot.set_noise(bearing_noise, steering_noise, distance_noise)
##
##print 'Robot:        ', myrobot
##print 'Measurements: ', myrobot.sense()
##


## IMPORTANT: You may uncomment the test cases below to test your code.
## But when you submit this code, your test cases MUST be commented
## out. Our testing program provides its own code for testing your
## sense function with randomized initial robot coordinates.
    


Robot:         [x=30.0 y=20.0 orient=0.0]
Measurements:  [6.004885648174475, 3.7295952571373605, 1.9295669970654687, 0.8519663271732721]


In [128]:
# --------------
# USER INSTRUCTIONS
#
# Now you will put everything together.
#
# First make sure that your sense and move functions
# work as expected for the test cases provided at the
# bottom of the previous two programming assignments.
# Once you are satisfied, copy your sense and move
# definitions into the robot class on this page, BUT
# now include noise.
#
# A good way to include noise in the sense step is to
# add Gaussian noise, centered at zero with variance
# of self.bearing_noise to each bearing. You can do this
# with the command random.gauss(0, self.bearing_noise)
#
# In the move step, you should make sure that your
# actual steering angle is chosen from a Gaussian
# distribution of steering angles. This distribution
# should be centered at the intended steering angle
# with variance of self.steering_noise.
#
# Feel free to use the included set_noise function.
#
# Please do not modify anything except where indicated
# below.

from math import *
import random

# --------
# 
# some top level parameters
#

max_steering_angle = pi / 4.0 # You do not need to use this value, but keep in mind the limitations of a real car.
bearing_noise = 0.1 # Noise parameter: should be included in sense function.
steering_noise = 0.1 # Noise parameter: should be included in move function.
distance_noise = 5.0 # Noise parameter: should be included in move function.

tolerance_xy = 15.0 # Tolerance for localization in the x and y directions.
tolerance_orientation = 0.25 # Tolerance for orientation.


# --------
# 
# the "world" has 4 landmarks.
# the robot's initial coordinates are somewhere in the square
# represented by the landmarks.
#
# NOTE: Landmark coordinates are given in (y, x) form and NOT
# in the traditional (x, y) format!

landmarks  = [[0.0, 100.0], [0.0, 0.0], [100.0, 0.0], [100.0, 100.0]] # position of 4 landmarks in (y, x) format.
world_size = 100.0 # world is NOT cyclic. Robot is allowed to travel "out of bounds"

# ------------------------------------------------
# 
# this is the robot class
#

class robot:

    # --------
    # init: 
    #    creates robot and initializes location/orientation 
    #

    def __init__(self, length = 20.0):
        self.x = random.random() * world_size # initial x position
        self.y = random.random() * world_size # initial y position
        self.orientation = random.random() * 2.0 * pi # initial orientation
        self.length = length # length of robot
        self.bearing_noise  = 0.0 # initialize bearing noise to zero
        self.steering_noise = 0.0 # initialize steering noise to zero
        self.distance_noise = 0.0 # initialize distance noise to zero

    # --------
    # set: 
    #    sets a robot coordinate
    #

    def set(self, new_x, new_y, new_orientation):

        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise ValueError, 'Orientation must be in [0..2pi]'
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)

    # --------
    # set_noise: 
    #    sets the noise parameters
    #
    def set_noise(self, new_b_noise, new_s_noise, new_d_noise):
        # makes it possible to change the noise parameters
        # this is often useful in particle filters
        self.bearing_noise  = float(new_b_noise)
        self.steering_noise = float(new_s_noise)
        self.distance_noise = float(new_d_noise)

    # --------
    # measurement_prob
    #    computes the probability of a measurement
    #  

    def measurement_prob(self, measurements):

        # calculate the correct measurement
        predicted_measurements = self.sense(0) # Our sense function took 0 as an argument to switch off noise.


        # compute errors
        error = 1.0
        for i in range(len(measurements)):
            error_bearing = abs(measurements[i] - predicted_measurements[i])
            error_bearing = (error_bearing + pi) % (2.0 * pi) - pi # truncate
            

            # update Gaussian
            error *= (exp(- (error_bearing ** 2) / (self.bearing_noise ** 2) / 2.0) /  
                      sqrt(2.0 * pi * (self.bearing_noise ** 2)))

        return error
    
    def __repr__(self): #allows us to print robot attributes.
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), 
                                                str(self.orientation))
    
    ############# ONLY ADD/MODIFY CODE BELOW HERE ###################
       
    # --------
    # move: 
    #   
    def move(self, motion):    
        epsilon = 0.0001
        x,y,theta = self.x,self.y,self.orientation
        alpha, d = motion

        if abs(alpha) > max_steering_angle:
            raise ValueError, "Exceeded max steering angle"
            
        if d < 0.0:
            raise ValueError, "Can't move backwards"

        alpha += random.gauss(0,self.steering_noise)
        d += random.gauss(0,self.distance_noise)

        beta = d / self.length * tan(alpha)

        
        if abs(beta) > epsilon:
            R = d / beta
            cx = x - sin(theta)*R
            cy = y + cos(theta)*R
            x = cx + sin(theta + beta)*R
            y = cy - cos(theta + beta)*R

        else:
            x = x + d * cos(theta)
            y = y + d * sin(theta)

        theta = (theta + beta) % (2*pi)
        result = robot(self.length)
        result.set(x,y,theta) 
        result.set_noise(self.bearing_noise, self.steering_noise, self.distance_noise)
        return result # make sure your move function returns an instance
                      # of the robot class with the correct coordinates.
   
    
    # copy your code from the previous exercise
    # and modify it so that it simulates motion noise
    # according to the noise parameters
    #           self.steering_noise
    #           self.distance_noise

    # --------
    # sense: 
    #    
    def sense(self,noise=True): 
        return [(
                atan2(
                    landmark[0]-self.y,
                    landmark[1]-self.x
                ) 
                - self.orientation 
                + (random.gauss(0.,self.bearing_noise) if noise else 0.)
                ) % (2*pi)
             for landmark in landmarks]        

    # copy your code from the previous exercise
    # and modify it so that it simulates bearing noise
    # according to
    #           self.bearing_noise

    ############## ONLY ADD/MODIFY CODE ABOVE HERE ####################

# --------
#
# extract position from a particle set
# 

def get_position(p):
    x = 0.0
    y = 0.0
    orientation = 0.0
    for i in range(len(p)):
        x += p[i].x
        y += p[i].y
        # orientation is tricky because it is cyclic. By normalizing
        # around the first particle we are somewhat more robust to
        # the 0=2pi problem
        orientation += (((p[i].orientation - p[0].orientation + pi) % (2.0 * pi)) 
                        + p[0].orientation - pi)
    return [x / len(p), y / len(p), orientation / len(p)]

# --------
#
# The following code generates the measurements vector
# You can use it to develop your solution.
# 


def generate_ground_truth(motions):

    myrobot = robot()
    myrobot.set_noise(bearing_noise, steering_noise, distance_noise)

    Z = []
    T = len(motions)

    for t in range(T):
        myrobot = myrobot.move(motions[t])
        Z.append(myrobot.sense())
    #print 'Robot:    ', myrobot
    return [myrobot, Z]

# --------
#
# The following code prints the measurements associated
# with generate_ground_truth
#

def print_measurements(Z):

    T = len(Z)

    print 'measurements = [[%.8s, %.8s, %.8s, %.8s],' % \
        (str(Z[0][0]), str(Z[0][1]), str(Z[0][2]), str(Z[0][3]))
    for t in range(1,T-1):
        print '                [%.8s, %.8s, %.8s, %.8s],' % \
            (str(Z[t][0]), str(Z[t][1]), str(Z[t][2]), str(Z[t][3]))
    print '                [%.8s, %.8s, %.8s, %.8s]]' % \
        (str(Z[T-1][0]), str(Z[T-1][1]), str(Z[T-1][2]), str(Z[T-1][3]))

# --------
#
# The following code checks to see if your particle filter
# localizes the robot to within the desired tolerances
# of the true position. The tolerances are defined at the top.
#

def check_output(final_robot, estimated_position):

    error_x = abs(final_robot.x - estimated_position[0])
    error_y = abs(final_robot.y - estimated_position[1])
    error_orientation = abs(final_robot.orientation - estimated_position[2])
    error_orientation = (error_orientation + pi) % (2.0 * pi) - pi
    correct = error_x < tolerance_xy and error_y < tolerance_xy \
              and error_orientation < tolerance_orientation
    return correct



def particle_filter(motions, measurements, N=500): # I know it's tempting, but don't change N!
    # --------
    #
    # Make particles
    # 

    p = []
    for i in range(N):
        r = robot()
        r.set_noise(bearing_noise, steering_noise, distance_noise)
        p.append(r)

    # --------
    #
    # Update particles
    #     

    for t in range(len(motions)):
    
        # motion update (prediction)
        p2 = []
        for i in range(N):
            p2.append(p[i].move(motions[t]))
        p = p2

        # measurement update
        w = []
        for i in range(N):
            w.append(p[i].measurement_prob(measurements[t]))

        # resampling
        p3 = []
        index = int(random.random() * N)
        beta = 0.0
        mw = max(w)
        for i in range(N):
            beta += random.random() * 2.0 * mw
            while beta > w[index]:
                beta -= w[index]
                index = (index + 1) % N
            p3.append(p[index])
        p = p3
    
    return get_position(p)

## IMPORTANT: You may uncomment the test cases below to test your code.
## But when you submit this code, your test cases MUST be commented
## out.
##
## You can test whether your particle filter works using the
## function check_output (see test case 2). We will be using a similar
## function. Note: Even for a well-implemented particle filter this
## function occasionally returns False. This is because a particle
## filter is a randomized algorithm. We will be testing your code
## multiple times. Make sure check_output returns True at least 80%
## of the time.


 
## --------
## TEST CASES:
## 
##1) Calling the particle_filter function with the following
##    motions and measurements should return a [x,y,orientation]
##    vector near [x=93.476 y=75.186 orient=5.2664], that is, the
##    robot's true location.
##
# motions = [[2. * pi / 10, 20.] for row in range(8)]
# measurements = [[4.746936, 3.859782, 3.045217, 2.045506],
#                [3.510067, 2.916300, 2.146394, 1.598332],
#                [2.972469, 2.407489, 1.588474, 1.611094],
#                [1.906178, 1.193329, 0.619356, 0.807930],
#                [1.352825, 0.662233, 0.144927, 0.799090],
#                [0.856150, 0.214590, 5.651497, 1.062401],
#                [0.194460, 5.660382, 4.761072, 2.471682],
#                [5.717342, 4.736780, 3.909599, 2.342536]]

# print particle_filter(motions, measurements)

## 2) You can generate your own test cases by generating
##    measurements using the generate_ground_truth function.
##    It will print the robot's last location when calling it.
##
##
# number_of_iterations = 6
# motions = [[2. * pi / 20, 12.] for row in range(number_of_iterations)]

# x = generate_ground_truth(motions)
# final_robot = x[0]
# measurements = x[1]
# estimated_position = particle_filter(motions, measurements)
# print_measurements(measurements)
# print 'Ground truth:    ', final_robot
# print 'Particle filter: ', estimated_position
# print 'Code check:      ', check_output(final_robot, estimated_position)





[112.21468940524844, 33.593244669802765, 5.113059358986888]
measurements = [[5.955335, 4.741661, 3.315040, 1.189972],
                [5.602441, 4.487142, 3.117210, 1.128018],
                [5.564562, 4.426053, 3.283101, 1.344245],
                [5.141196, 3.968924, 2.866150, 1.506575],
                [4.448817, 3.660606, 2.798613, 1.686329],
                [4.364213, 3.518588, 2.500615, 1.507034]]
Ground truth:     [x=114.17 y=62.979 orient=0.2249]
Particle filter:  [110.38936314358023, 52.969110132890485, 0.16610785898480446]
Code check:       True


# Robot Motion - A* algorithm

In [221]:
# ----------
# User Instructions:
# 
# Define a function, search() that returns a list
# in the form of [optimal path length, row, col]. For
# the grid shown below, your function should output
# [11, 4, 5].
#
# If there is no valid path from the start point
# to the goal, your function should return the string
# 'fail'
# ----------

# Grid format:
#   0 = Navigable space
#   1 = Occupied space


grid = [[0, 1, 1, 0, 0, 0],
        [0, 1, 1, 0, 0, 0],
        [0, 1, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]]

heuristic = [[9, 8, 7, 6, 5, 4],
             [8, 7, 6, 5, 4, 3],
             [7, 6, 5, 4, 3, 2],
             [6, 5, 4, 3, 2, 1],
             [5, 4, 3, 2, 1, 0]]

grid = [[0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0]]
heuristic = [[9, 8, 7, 6, 5, 4],
             [8, 7, 6, 5, 4, 3],
             [7, 6, 5, 4, 3, 2],
             [6, 5, 4, 3, 2, 1],
             [5, 4, 3, 2, 1, 0]]




init = [0, 0]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1

delta = [[-1, 0], # go up
         [ 0,-1], # go left
         [ 1, 0], # go down
         [ 0, 1]] # go right

delta_name = ['^', '<', 'v', '>']

def search(grid,init,goal,cost,heuristic):
    init = tuple(init)
    goal = tuple(goal)
    
    def in_bounds(y,x):
        return y >= 0 and y < len(grid) and x >= 0 and x < len(grid[0])
    
    def ptable(t):
        for r in t:
            print(r)
        print('')

    open_list = [(0,init,init)]
    closed = [[0 for _ in range(len(grid[0]))] for _ in range(len(grid))]
    expand = [[-1 for _ in range(len(grid[0]))] for _ in range(len(grid))]
    previous = {}
    step = 0
    done = False
    h = lambda z: heuristic[z[0]][z[1]]
    while not done and open_list:   
        open_list = sorted(open_list, key=lambda v: v[0]+h(v[1]))
        [curr], open_list = open_list[:1], open_list[1:]
        val,pos,prev = curr
        if closed[pos[0]][pos[1]]:
            continue
        closed[pos[0]][pos[1]] = val+cost
        expand[pos[0]][pos[1]] = step
        previous[pos] = prev
        step += 1
        
        for d in delta:
            y,x = (pos[0]+d[0],pos[1]+d[1])
            if in_bounds(y,x):
                if grid[y][x] == 0 and closed[y][x] == 0:                    
                    open_list.append((val + cost, (y,x), pos))
                    if goal == (y,x):
                        done = True
                        expand[y][x] = step                        
                        previous[goal] = pos

    if not done:
        return 'no path'
    
    path = [[' ' for _ in range(len(grid[0]))] for _ in range(len(grid))]
    path[goal[0]][goal[1]] = '*'
    curr = goal
    direction = {
        (-1,0): '^',
        (1,0): 'v',
        (0,-1): '<',
        (0,1): '>'
    }

    while curr != init:
        prev = previous[curr]
        d = curr[0]-prev[0], curr[1]-prev[1]
        path[prev[0]][prev[1]] = direction[d]
        curr = prev

    return expand

In [222]:
search(grid,init,goal,cost, heuristic)

[[0, 2, 4, 6, 8, 9],
 [1, -1, -1, -1, -1, 10],
 [3, -1, -1, -1, -1, 11],
 [5, -1, -1, -1, -1, 12],
 [7, -1, -1, -1, -1, 13]]

# Dynamic Programming

In [24]:
# ----------
# User Instructions:
# 
# Create a function compute_value which returns
# a grid of values. The value of a cell is the minimum
# number of moves required to get from the cell to the goal. 
#
# If a cell is a wall or it is impossible to reach the goal from a cell,
# assign that cell a value of 99.
# ----------

grid = [[0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 1, 0]]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1 # the cost associated with moving from a cell to an adjacent one

delta = [[-1, 0 ], # go up
         [ 0, -1], # go left
         [ 1, 0 ], # go down
         [ 0, 1 ]] # go right

delta_name = ['^', '<', 'v', '>']

def compute_value(grid,goal,cost):
    # helpers
    def in_bounds(z):
        return z[0] >= 0 and z[0] < len(grid) and z[1] >= 0 and z[1] < len(grid[0])
    
    def ptable(t):
        for r in t:
            print(r)
        print('')
        
    def addpt(x,y):
        return x[0]+y[0], x[1]+y[1]

    goal = tuple(goal)
    q = [[99 for c in row] for row in grid]
    q[goal[0]][goal[1]] = 0

    open_list = [[0,goal]]
    ct = 0
    while open_list:
        ct+=1
        [exp], open_list = open_list[:1],open_list[1:]
        g,x = exp
        for d in delta:
            y = addpt(x,d)
            if not in_bounds(y) or grid[y[0]][y[1]]==1:
                continue
            f = g+cost
            if q[y[0]][y[1]] > f:
                q[y[0]][y[1]] = f 
                open_list.append([f,y])

#         open_list = sorted(open_list, key=lambda v:v[0])        
            
    
    # make sure your function returns a grid of values as 
    # demonstrated in the previous video.

    return q 


In [25]:
compute_value(grid,goal,cost)

25


[[9, 99, 7, 6, 5, 4],
 [8, 7, 6, 5, 4, 3],
 [9, 99, 5, 4, 3, 2],
 [10, 99, 4, 3, 2, 1],
 [11, 99, 5, 4, 99, 0]]

In [45]:
# ----------
# User Instructions:
# 
# Create a function compute_value which returns
# a grid of values. The value of a cell is the minimum
# number of moves required to get from the cell to the goal. 
#
# If a cell is a wall or it is impossible to reach the goal from a cell,
# assign that cell a value of 99.
# ----------

grid = [[0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 1, 0]]
goal = [len(grid)-1, len(grid[0])-1]
cost = 1 # the cost associated with moving from a cell to an adjacent one

delta = [[-1, 0 ], # go up
         [ 0, -1], # go left
         [ 1, 0 ], # go down
         [ 0, 1 ]] # go right

delta_name = ['^', '<', 'v', '>']

def optimal_policy(grid,goal,cost):
    # helpers
    def in_bounds(z):
        return z[0] >= 0 and z[0] < len(grid) and z[1] >= 0 and z[1] < len(grid[0])
    
    def ptable(t):
        for r in t:
            print(r)
        print('')
        
    def addpt(x,y):
        return x[0]+y[0], x[1]+y[1]

    goal = tuple(goal)
    q = [[99 for c in row] for row in grid]
    q[goal[0]][goal[1]] = 0
    p = [[' ' for c in row] for row in grid]
    p[goal[0]][goal[1]] = '*'
    open_list = [[0,goal]]
    delta_name = ['v','>','^','<']

    while open_list:
        [exp], open_list = open_list[:1],open_list[1:]
        g,x = exp
        for d,glyph in zip(delta,delta_name):
            y = addpt(x,d)
            if not in_bounds(y) or grid[y[0]][y[1]]==1:
                continue
            f = g+cost
            if q[y[0]][y[1]] > f:
                q[y[0]][y[1]] = f
                p[y[0]][y[1]] = glyph
                open_list.append([f,y])

    return q,p 


In [46]:
optimal_policy(grid,goal,cost)

([[9, 99, 7, 6, 5, 4],
  [8, 7, 6, 5, 4, 3],
  [9, 99, 5, 4, 3, 2],
  [10, 99, 4, 3, 2, 1],
  [11, 99, 5, 4, 99, 0]],
 [['v', ' ', '>', '>', '>', 'v'],
  ['>', '>', '>', '>', '>', 'v'],
  ['^', ' ', '>', '>', '>', 'v'],
  ['^', ' ', '>', '>', '>', 'v'],
  ['^', ' ', '^', '^', ' ', '*']])